In [1]:
import pandas as pd
import numpy as np
import itertools
import random
import math
from collections import defaultdict
from collections import Counter
import itertools
from matplotlib import pyplot as plt

# Reading data

In [2]:
alunos = pd.read_csv('alunos.csv', sep=';')
alunos.head()

,Data,Idade,Doenca
0,21/9/2019,16,sarampo
1,19/8/2006,18,meningite
2,15/1/2011,16,meningite
3,13/2/2009,17,catapora
4,19/6/2006,18,meningite


In [3]:
def pre_processing(path):
    students = pd.read_csv(path, sep=';')
    split_data = students.copy()
    complemento = students['Data'].str.split('/', expand=True)
    split_data['Dia'], split_data['Mes'], split_data['Ano'] = complemento[0], complemento[1], complemento[2]
    decades = [year[:-1]+'0' for year in split_data['Ano']]
    split_data['Decada'] = decades
    
    split_data['FaixaEtaria'] = [None]*len(split_data)
    bins = [int(split_data['Idade'].min())-1, int(split_data['Idade'].min())+2, int(split_data['Idade'].max())+1]
    labels = [str(bins[i-1])+'-'+str(bins[i]-1) for i in range(1, len(bins))]
    split_data['FaixaEtaria'] = pd.cut(split_data['Idade'], bins=bins, labels=labels)
    
    split_data['RangeMaximo'] = [str(split_data['Idade'].min())+'-'+str(split_data['Idade'].max())]*len(split_data)
    
    split_data = split_data[['Dia', 'Mes', 'Ano', 'Decada', 'Idade', 'FaixaEtaria', 'RangeMaximo', 'Doenca']]
    
    return students, split_data

### Separating data to generalize it when necessary

In [4]:
original_data, split_data = pre_processing('alunos.csv')

In [5]:
split_data

,Dia,Mes,Ano,Decada,Idade,FaixaEtaria,RangeMaximo,Doenca
0,21,9,2019,2010,16,15-17,16-20,sarampo
1,19,8,2006,2000,18,15-17,16-20,meningite
2,15,1,2011,2010,16,15-17,16-20,meningite
3,13,2,2009,2000,17,15-17,16-20,catapora
4,19,6,2006,2000,18,15-17,16-20,meningite
...,...,...,...,...,...,...,...,...
295,8,10,2017,2010,17,15-17,16-20,sarampo
296,21,9,2005,2000,19,18-20,16-20,catapora
297,29,5,2005,2000,16,15-17,16-20,sarampo
298,19,1,2007,2000,19,18-20,16-20,sarampo


In [6]:
students = split_data[['Dia','Mes','Ano','Idade','Doenca']]
students

,Dia,Mes,Ano,Idade,Doenca
0,21,9,2019,16,sarampo
1,19,8,2006,18,meningite
2,15,1,2011,16,meningite
3,13,2,2009,17,catapora
4,19,6,2006,18,meningite
...,...,...,...,...,...
295,8,10,2017,17,sarampo
296,21,9,2005,19,catapora
297,29,5,2005,16,sarampo
298,19,1,2007,19,sarampo


In [7]:
def initial_clusters(dataset, k):   
    def priority(dataset):
        generalization_order = [list(dataset.columns[i:-1]) for i in range(len(dataset.columns)-1)]
        generalization_order = [dataset.groupby(generalization_order[i]).groups for i in range(len(generalization_order))]

        return generalization_order
    
    clusters = defaultdict(list)
    clustered = set()
    generalization_order = priority(dataset)

    for i in range(len(generalization_order)):
        for key, value in zip(generalization_order[i].keys(), generalization_order[i].values()):
            if len(value) >= k and len(clustered.intersection(value)) == 0:
                # se o grupo do groupby é maior ou igual a k e nenhum dos valores já estiver agrupado
                clusters[key] = list(itertools.chain(value))
                clustered = clustered.union(clusters[key])
                
    return clusters, clustered

In [8]:
clusters, clustered = initial_clusters(students, 2)
clusters

defaultdict(list,
            {('5', '4', '2011', 18): [111, 212],
             ('1', '2011', 16): [2, 193],
             ('1', '2014', 20): [109, 275],
             ('1', '2015', 19): [178, 214],
             ('10', '2006', 19): [40, 209],
             ('10', '2012', 19): [175, 242, 299],
             ('11', '2007', 19): [15, 161],
             ('11', '2017', 16): [62, 119],
             ('11', '2019', 19): [21, 88],
             ('2', '2005', 18): [38, 55],
             ('2', '2006', 16): [29, 136],
             ('2', '2009', 17): [3, 248],
             ('2', '2012', 19): [60, 118],
             ('2', '2015', 20): [87, 102],
             ('3', '2007', 17): [154, 231],
             ('3', '2007', 19): [47, 155],
             ('3', '2013', 18): [148, 282],
             ('3', '2015', 17): [112, 254],
             ('3', '2017', 18): [100, 270],
             ('3', '2018', 16): [9, 96],
             ('4', '2005', 18): [67, 217],
             ('4', '2009', 20): [77, 210],
             ('4', 

In [9]:
def generalization_trees():
    date_lvl = {
        ('Dia', 'Mes', 'Ano'): 1,
        ('Mes', 'Ano'): 2,
        ('Ano'): 3,
        ('Decada'): 4
    }
    
    age_lvl = {
        ('Idade'): 1,
        ('FaixaEtaria'): 2,
        ('RangeMaximo'): 3
    }
    
    return date_lvl, age_lvl

In [10]:
gen_trees = generalization_trees()
periods_date = list(gen_trees[0].keys())
periods_age = list(gen_trees[1].keys())

all_combinations = list(itertools.product(periods_date, periods_age))
print(all_combinations)
order = []
for i in all_combinations:
    order.append(gen_trees[0][i[0]]+gen_trees[1][i[1]])
print(order)
list(zip(order, all_combinations))
#[x for y, x in sorted(zip(order, all_combinations))]

[(('Dia', 'Mes', 'Ano'), 'Idade'), (('Dia', 'Mes', 'Ano'), 'FaixaEtaria'), (('Dia', 'Mes', 'Ano'), 'RangeMaximo'), (('Mes', 'Ano'), 'Idade'), (('Mes', 'Ano'), 'FaixaEtaria'), (('Mes', 'Ano'), 'RangeMaximo'), ('Ano', 'Idade'), ('Ano', 'FaixaEtaria'), ('Ano', 'RangeMaximo'), ('Decada', 'Idade'), ('Decada', 'FaixaEtaria'), ('Decada', 'RangeMaximo')]
[2, 3, 4, 3, 4, 5, 4, 5, 6, 5, 6, 7]


[(2, (('Dia', 'Mes', 'Ano'), 'Idade')),
 (3, (('Dia', 'Mes', 'Ano'), 'FaixaEtaria')),
 (4, (('Dia', 'Mes', 'Ano'), 'RangeMaximo')),
 (3, (('Mes', 'Ano'), 'Idade')),
 (4, (('Mes', 'Ano'), 'FaixaEtaria')),
 (5, (('Mes', 'Ano'), 'RangeMaximo')),
 (4, ('Ano', 'Idade')),
 (5, ('Ano', 'FaixaEtaria')),
 (6, ('Ano', 'RangeMaximo')),
 (5, ('Decada', 'Idade')),
 (6, ('Decada', 'FaixaEtaria')),
 (7, ('Decada', 'RangeMaximo'))]

In [11]:
def sort_priority(generalizations):
    date_tree, age_tree = 0()
    retur1n
sort_priority

<>:2: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:2: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
C:\Users\italo.abreu\AppData\Local\Temp\ipykernel_7052\529450774.py:2: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  date_tree, age_tree = 0()


<function __main__.sort_priority(generalizations)>

In [13]:
priority_level(split_data)

NameError: name 'priority_level' is not defined

In [14]:
def sort_priority():
    return

def priority_level(dataset):
    gen_trees = generalization_trees()
    periods_date = list(gen_trees[0].keys())
    periods_age = list(gen_trees[1].keys())

    all_combinations = list(itertools.product(periods_date, periods_age))
    all_combinations = [list(itertools.chain.from_iterable([[i] if(type(i) == str) else list(i) for i in list_i ])) \
                        for list_i in all_combinations] 
    all_combinations = [i for i in all_combinations if('FaixaEtaria' in i or 'RangeMaximo' in i or 'Decada' in i)]
    print(all_combinations)
    all_groups = [dataset.groupby(all_combinations[i]).groups for i in range(len(all_combinations))]

    return all_groups

def cluster_by_priority(dataset, clusters, clustered, k):
    not_clustered = list(set(dataset.index) - set(clustered))
    not_clustered_df = dataset.loc[not_clustered]
    groups = priority_level(not_clustered_df)

    for i in range(len(groups)):
        for key, value in zip(groups[i].keys(), groups[i].values()):
            if (len(value) - len(clustered.intersection(value))) >= k : # and len(clustered.intersection(value)) == 0:
                # se o grupo do groupby é maior ou igual a k e nenhum dos valores já estiver agrupado
                clusters[key] = list(itertools.chain(list(set(value) - set(clustered))))
                clustered = clustered.union(clusters[key])
                
    return clusters, len(dataset) - len(clustered), list(set(dataset.index) - set(clustered))

cluster_by_priority(split_data, clusters, clustered, 2)

[['Dia', 'Mes', 'Ano', 'FaixaEtaria'], ['Dia', 'Mes', 'Ano', 'RangeMaximo'], ['Mes', 'Ano', 'FaixaEtaria'], ['Mes', 'Ano', 'RangeMaximo'], ['Ano', 'FaixaEtaria'], ['Ano', 'RangeMaximo'], ['Decada', 'Idade'], ['Decada', 'FaixaEtaria'], ['Decada', 'RangeMaximo']]


(defaultdict(list,
             {('5', '4', '2011', 18): [111, 212],
              ('1', '2011', 16): [2, 193],
              ('1', '2014', 20): [109, 275],
              ('1', '2015', 19): [178, 214],
              ('10', '2006', 19): [40, 209],
              ('10', '2012', 19): [175, 242, 299],
              ('11', '2007', 19): [15, 161],
              ('11', '2017', 16): [62, 119],
              ('11', '2019', 19): [21, 88],
              ('2', '2005', 18): [38, 55],
              ('2', '2006', 16): [29, 136],
              ('2', '2009', 17): [3, 248],
              ('2', '2012', 19): [60, 118],
              ('2', '2015', 20): [87, 102],
              ('3', '2007', 17): [154, 231],
              ('3', '2007', 19): [47, 155],
              ('3', '2013', 18): [148, 282],
              ('3', '2015', 17): [112, 254],
              ('3', '2017', 18): [100, 270],
              ('3', '2018', 16): [9, 96],
              ('4', '2005', 18): [67, 217],
              ('4', '2009', 20): [77, 21

In [15]:
split_data.loc[[17, 146, 140, 127]]

,Dia,Mes,Ano,Decada,Idade,FaixaEtaria,RangeMaximo,Doenca
17,9,7,2010,2010,19,18-20,16-20,meningite
146,23,11,2007,2000,20,18-20,16-20,catapora
140,7,6,2007,2000,20,18-20,16-20,meningite
127,14,11,2010,2010,19,18-20,16-20,gripe


In [16]:
def clustering_remaining_tuples(dataset, clusters, clustered, k): # dataset split with age range
    def priority_level(dataset, age_level, date_level):
        if(age_level == 2 and date_level < 4):
            data = dataset[['Dia', 'Mes', 'Ano', 'FaixaEtaria', 'Doenca']]
            generalization_order = [list(dataset.columns[i:-1]) for i in range(len(data.columns[:-1]))]
            generalization_order = [dataset.groupby(generalization_order[i]).groups for i in range(len(generalization_order))]
        
            return generalization_order
    
    # Agrupar por década e intervalo de idade
    not_clustered = list(set(dataset.index) - set(clustered))
    not_clustered_df = dataset.loc[not_clustered]
    
    
    
    return not_clustered_df
    
clustering_remaining_tuples(split_data, clusters, clustered, 2)

,Dia,Mes,Ano,Decada,Idade,FaixaEtaria,RangeMaximo,Doenca
4,19,6,2006,2000,18,15-17,16-20,meningite
5,18,5,2010,2010,20,18-20,16-20,meningite
7,5,9,2013,2010,16,15-17,16-20,gripe
11,24,6,2009,2000,19,18-20,16-20,meningite
18,27,5,2005,2000,18,15-17,16-20,gripe
...,...,...,...,...,...,...,...,...
281,17,4,2008,2000,18,15-17,16-20,catapora
286,28,11,2014,2010,18,15-17,16-20,sarampo
289,1,6,2014,2010,16,15-17,16-20,catapora
293,6,11,2007,2000,18,15-17,16-20,meningite


In [17]:
list(range(5))

[0, 1, 2, 3, 4]

# k-anonimato

In [18]:
def k_anonymity(dataset, clusters):
    k_anonymized = pd.DataFrame([[None]*3]*len(dataset), columns=dataset.columns)
    for key, value in zip(clusters.keys(), clusters.values()):
        if not isinstance(key, list) and isinstance(key, str):
            if '-' not in key:
                k_anonymized.loc[value] = [[key, '{}-{}'.format(dataset['Idade'].min(), dataset['Idade'].max()), key]+[dataset[dataset.columns[-1]].loc[v]] for v in value]
            else:
                k_anonymized.loc[value] = [['*', key]+[dataset[dataset.columns[-1]].loc[v]] for v in value]

        if not isinstance(key, list) and isinstance(key, int):
            k_anonymized.loc[value] = [['*', key]+[dataset[dataset.columns[-1]].loc[v]] for v in value]

        elif all(map(lambda x: isinstance(x, str), key)):
            k_anonymized.loc[value] = [[k for k in key]+[dataset[dataset.columns[-1]].loc[v]] for v in value]

        else:
            to_fill = [key[0]]
            for i in range(1, len(key)):
                if type(key[i]) == str:
                    to_fill[0] = to_fill[0] + str('/'+key[i])
                else:
                    to_fill.append(key[i])
            k_anonymized.loc[value] = [to_fill+[dataset[dataset.columns[-1]].loc[v]] for v in value]

    return k_anonymized

In [19]:
def plot_histogram(clusters):
    lengths = []
    for cluster in clusters.values():
        lengths.append(len(cluster))

    counter = dict(sorted(Counter(lengths).items()))
    keys_str = [str(key) for key in counter.keys()]
    plt.bar(keys_str, counter.values())
    
    return plt.show()

In [21]:
def calc_distance(x, y, attributes):
    D = 0
    for attr in attributes:
        if(x[attr] != y[attr]):
            D += 1
        print(attr, ': ', D, ' - ', x[attr], ' ?= ', y[attr])
            
    return D

In [22]:
# testing calc_distancabse
calc_distance(split_data.iloc[7], split_data.iloc[17], ['Dia', 'Mes', 'Ano', 'Decada', 'Idade', 'FaixaEtaria', 'RangeMaximo'])  


Dia :  1  -  5  ?=  9
Mes :  2  -  9  ?=  7
Ano :  3  -  2013  ?=  2010
Decada :  3  -  2010  ?=  2010
Idade :  4  -  16  ?=  19
FaixaEtaria :  5  -  15-17  ?=  18-20
RangeMaximo :  5  -  16-20  ?=  16-20


5

In [23]:
split_data.loc[[7,17]]

,Dia,Mes,Ano,Decada,Idade,FaixaEtaria,RangeMaximo,Doenca
7,5,9,2013,2010,16,15-17,16-20,gripe
17,9,7,2010,2010,19,18-20,16-20,meningite


# Chamadas de funções

In [ ]:
clusters, clustered = initial_clusters(alunos_copy, 20)
all_clusters = clustering_remaining_tuples(alunos_copy, clusters, clustered, 20)

In [ ]:
a = k_anonymity(alunos, all_clusters)
a.head()

In [ ]:
plot_histogram(all_clusters)